In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import importlib
from datetime import datetime

In [2]:
import Hypers
importlib.reload(Hypers)
import utils
importlib.reload(utils)
import RatingSet
importlib.reload(RatingSet)
import PredictorModel
importlib.reload(PredictorModel)
import Trainer
importlib.reload(Trainer)

utils.fix_random_seed(Hypers.Config.seed)

In [3]:
# Optional: Split data into train and test
merged_dict = utils.load_pickle(os.path.join(Hypers.Config.data_path, "features_normalized_ratings_extended.pkl"))
train_dict, test_dict = utils.spilt_train_valid(merged_dict, random_select=True, save=True)
print(len(train_dict), len(test_dict))

0 0


In [ ]:
model = PredictorModel.PredictorModel(input_size=len(Hypers.feature_list), 
                                      hidden_size=Hypers.Config.hidden_size, 
                                      proj_size=Hypers.Config.proj_size)
criterion = nn.MSELoss(reduction="sum")
optimizer = torch.optim.Adam(model.parameters(), lr=Hypers.Config.learning_rate)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dict = utils.load_pickle(os.path.join(Hypers.Config.data_path, "train_dict.pkl"))
test_dict = utils.load_pickle(os.path.join(Hypers.Config.data_path, "test_dict.pkl"))
print(len(train_dict), len(test_dict))

In [ ]:
train_set, test_set = RatingSet.RatingSet(train_dict), RatingSet.RatingSet(test_dict)
train_loader = DataLoader(train_set, batch_size=Hypers.Config.batch_size, shuffle=True, collate_fn=utils.custom_collate_fn)
test_loader = DataLoader(test_set, batch_size=Hypers.Config.batch_size, shuffle=True, collate_fn=utils.custom_collate_fn)

trainer = Trainer.Trainer(model, criterion, optimizer, device, train_loader, test_loader)
train_loss, train_accuracy, test_loss, test_accuracy = trainer.train_loop()

In [ ]:
utils.plot_graph(train_loss, train_accuracy, test_loss, test_accuracy)